In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
file_path = 'dataset.csv'
df = pd.read_csv(file_path)
df = df.drop(df.columns[-2:], axis=1)
df.head()

,date,channel_name,campaign_name,creative_id,device_type,cost_usd,impressions,clicks,signups,payments,ctr,cr(click-signup),cr(signup-payment)
0,12/6/2024,Google Search Engine,Awareness Female 35-44,Creative_32,Android,2.335280,719,38,3,1,5.29%,7.89%,33.33%
1,27/1/2024,YouTube,Remarketing Male 55+,Creative_12,iOS,4.703972,4166,15,1,1,0.36%,6.67%,100.00%
2,9/1/2024,Google Search Engine,Awareness Male 45-54,Creative_30,iOS,3.018451,1188,25,2,1,2.10%,8.00%,50.00%
3,7/1/2024,Instagram Stories,Remarketing Male 55+,Creative_54,Android,0.760292,1292,7,0,0,0.54%,0.00%,0.00%
4,30/5/2024,Google Search Engine,Branding Male 18-24,Creative_98,Desktop,1.167750,700,9,0,0,1.29%,0.00%,0.00%


In [3]:
df.dtypes

date                   object
channel_name           object
campaign_name          object
creative_id            object
device_type            object
cost_usd              float64
impressions             int64
clicks                  int64
signups                 int64
payments                int64
ctr                    object
cr(click-signup)       object
cr(signup-payment)     object
dtype: object

In [4]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

percentage_columns = ['ctr', 'cr(click-signup)', 'cr(signup-payment)']
for column in percentage_columns:
    # Strip the '%' sign and convert to float
    df[column] = df[column].str.rstrip('%').astype(float)/100

In [5]:
df.head()

,date,channel_name,campaign_name,creative_id,device_type,cost_usd,impressions,clicks,signups,payments,ctr,cr(click-signup),cr(signup-payment)
0,2024-06-12,Google Search Engine,Awareness Female 35-44,Creative_32,Android,2.335280,719,38,3,1,0.0529,0.0789,0.3333
1,2024-01-27,YouTube,Remarketing Male 55+,Creative_12,iOS,4.703972,4166,15,1,1,0.0036,0.0667,1.0000
2,2024-01-09,Google Search Engine,Awareness Male 45-54,Creative_30,iOS,3.018451,1188,25,2,1,0.0210,0.0800,0.5000
3,2024-01-07,Instagram Stories,Remarketing Male 55+,Creative_54,Android,0.760292,1292,7,0,0,0.0054,0.0000,0.0000
4,2024-05-30,Google Search Engine,Branding Male 18-24,Creative_98,Desktop,1.167750,700,9,0,0,0.0129,0.0000,0.0000


In [6]:
channel_summary = df.groupby('channel_name')[['cost_usd', 'impressions','clicks','signups','payments']].sum().reset_index()
channel_summary['CPM'] = (channel_summary['cost_usd'] / channel_summary['impressions']) * 1000
channel_summary['CPC'] = channel_summary['cost_usd'] / channel_summary['clicks']
channel_summary['CPS'] = channel_summary['cost_usd'] / channel_summary['signups']
channel_summary['CPP'] = channel_summary['cost_usd'] / channel_summary['payments']
channel_summary = channel_summary.rename(columns={'cost_usd': 'total_cost','impressions': 'total_impressions','clicks':'total_clicks'})
channel_summary

,channel_name,total_cost,total_impressions,total_clicks,signups,payments,CPM,CPC,CPS,CPP
0,Facebook News Feed,2139.618646,535341,15124,869,654,3.996740,0.141472,2.462162,3.271588
1,Google Search Engine,4836.019245,1068680,44502,3196,1359,4.525227,0.108670,1.513147,3.558513
2,Instagram Stories,1118.888303,344640,6565,294,289,3.246542,0.170432,3.805743,3.871586
3,MoPub,559.119707,202883,1591,0,0,2.755873,0.351427,inf,inf
4,TikTok,532.051920,233068,2039,2,2,2.282818,0.260938,266.025960,266.025960
5,YouTube,1225.759759,383111,6216,211,210,3.199490,0.197194,5.809288,5.836951


In [95]:
channel_summary.to_csv('channel_summary.csv', index=False)

In [7]:
split_columns = df['campaign_name'].str.split(' ', 2, expand=True)
split_columns.columns = ['objective', 'gender', 'age']
expanded_df = df.join(split_columns)
expanded_df.head()

C:\Users\LYX\AppData\Local\Temp\ipykernel_10316\3733338905.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  split_columns = df['campaign_name'].str.split(' ', 2, expand=True)


,date,channel_name,campaign_name,creative_id,device_type,cost_usd,impressions,clicks,signups,payments,ctr,cr(click-signup),cr(signup-payment),objective,gender,age
0,2024-06-12,Google Search Engine,Awareness Female 35-44,Creative_32,Android,2.335280,719,38,3,1,0.0529,0.0789,0.3333,Awareness,Female,35-44
1,2024-01-27,YouTube,Remarketing Male 55+,Creative_12,iOS,4.703972,4166,15,1,1,0.0036,0.0667,1.0000,Remarketing,Male,55+
2,2024-01-09,Google Search Engine,Awareness Male 45-54,Creative_30,iOS,3.018451,1188,25,2,1,0.0210,0.0800,0.5000,Awareness,Male,45-54
3,2024-01-07,Instagram Stories,Remarketing Male 55+,Creative_54,Android,0.760292,1292,7,0,0,0.0054,0.0000,0.0000,Remarketing,Male,55+
4,2024-05-30,Google Search Engine,Branding Male 18-24,Creative_98,Desktop,1.167750,700,9,0,0,0.0129,0.0000,0.0000,Branding,Male,18-24


In [8]:
pivot_table = expanded_df.groupby(['objective', 'channel_name', 'device_type'])[['cost_usd','impressions', 'clicks', 'signups', 'payments']].sum().reset_index()
pivot_table

,objective,channel_name,device_type,cost_usd,impressions,clicks,signups,payments
0,Awareness,Facebook News Feed,Android,237.677213,63958,1772,114,82
1,Awareness,Facebook News Feed,Desktop,61.017573,11505,373,4,4
2,Awareness,Facebook News Feed,iOS,201.270655,50651,1452,95,74
3,Awareness,Google Search Engine,Android,508.990768,107483,4785,358,138
4,Awareness,Google Search Engine,Desktop,132.214653,30180,1154,44,41
...,...,...,...,...,...,...,...,...
67,Remarketing,TikTok,Desktop,18.218244,6475,62,0,0
68,Remarketing,TikTok,iOS,51.888233,23173,198,0,0
69,Remarketing,YouTube,Android,145.389652,42877,764,32,32
70,Remarketing,YouTube,Desktop,33.789478,22849,151,0,0


In [53]:
pivot_table['CPM'] = (pivot_table['cost_usd'] / pivot_table['impressions']) * 1000
pivot_table['CTR'] = (pivot_table['clicks'] / pivot_table['impressions']) 
pivot_table['CPC'] = (pivot_table['cost_usd'] / pivot_table['clicks']) 
pivot_table['CPS'] = (pivot_table['cost_usd'] / pivot_table['signups']) 
pivot_table['CPP'] = (pivot_table['cost_usd'] / pivot_table['payments']) 
pivot_table['cr2'] = (pivot_table['signups'] / pivot_table['clicks'])
pivot_table['cr3'] = (pivot_table['payments'] / pivot_table['signups'])
pivot_table

,objective,channel_name,device_type,cost_usd,impressions,clicks,signups,payments,CPM,CTR,CPC,CPS,CPP,cr2,cr3
0,Awareness,Facebook News Feed,Android,237.677213,63958,1772,114,82,3.716145,0.027706,0.134129,2.084888,2.898503,0.064334,0.719298
1,Awareness,Facebook News Feed,Desktop,61.017573,11505,373,4,4,5.303570,0.032421,0.163586,15.254393,15.254393,0.010724,1.000000
2,Awareness,Facebook News Feed,iOS,201.270655,50651,1452,95,74,3.973676,0.028667,0.138616,2.118638,2.719874,0.065427,0.778947
3,Awareness,Google Search Engine,Android,508.990768,107483,4785,358,138,4.735547,0.044519,0.106372,1.421762,3.688339,0.074817,0.385475
4,Awareness,Google Search Engine,Desktop,132.214653,30180,1154,44,41,4.380870,0.038237,0.114571,3.004878,3.224748,0.038128,0.931818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Remarketing,TikTok,Desktop,18.218244,6475,62,0,0,2.813628,0.009575,0.293843,inf,inf,0.000000,NaN
68,Remarketing,TikTok,iOS,51.888233,23173,198,0,0,2.239168,0.008544,0.262062,inf,inf,0.000000,NaN
69,Remarketing,YouTube,Android,145.389652,42877,764,32,32,3.390854,0.017818,0.190301,4.543427,4.543427,0.041885,1.000000
70,Remarketing,YouTube,Desktop,33.789478,22849,151,0,0,1.478817,0.006609,0.223771,inf,inf,0.000000,NaN


In [38]:
melted_df = pivot_table.melt(id_vars=['objective', 'channel_name', 'device_type'], 
                     value_vars=['impressions', 'clicks', 'signups', 'payments'],
                     var_name='metric',
                     value_name='value')
melted_df

,objective,channel_name,device_type,metric,value
0,Awareness,Facebook News Feed,Android,impressions,63958
1,Awareness,Facebook News Feed,Desktop,impressions,11505
2,Awareness,Facebook News Feed,iOS,impressions,50651
3,Awareness,Google Search Engine,Android,impressions,107483
4,Awareness,Google Search Engine,Desktop,impressions,30180
...,...,...,...,...,...
283,Remarketing,TikTok,Desktop,payments,0
284,Remarketing,TikTok,iOS,payments,0
285,Remarketing,YouTube,Android,payments,32
286,Remarketing,YouTube,Desktop,payments,0


In [40]:
fig = px.bar(
    melted_df, 
    x='metric', 
    y='value', 
    color='channel_name',
    facet_col='objective',
    text='value',
    title='Metrics by Channel and Objective',
    labels={'value': 'Count'},
    height=800,
    width=1200
)

# Remove 'objective=' from facet titles
fig.update_layout(
    title_text='Metrics by Channel and Objective',
    title_x=0.5,  # Center the title
    title_font_size=20
)

# Update text position and size
fig.update_traces(
    texttemplate='%{text:.0f}',  # Show number without decimal places
    textposition='inside',  # Place text inside bars
    textfont_size=10  # Adjust text size
)

fig.show()

In [69]:
pivot_nodevice = expanded_df.groupby(['objective', 'channel_name'])[['cost_usd','impressions', 'clicks', 'signups', 'payments']].sum().reset_index()
pivot_noobjective = expanded_df.groupby(['device_type', 'channel_name'])[['cost_usd','impressions', 'clicks', 'signups', 'payments']].sum().reset_index()
campaign_summary = expanded_df.groupby(['objective'])[['cost_usd','impressions', 'clicks', 'signups', 'payments']].sum().reset_index()


In [ ]:
pivot_noobjective['CPM'] = (pivot_noobjective['cost_usd'] / pivot_noobjective['impressions']) * 1000
pivot_noobjective['CTR'] = (pivot_noobjective['clicks'] / pivot_noobjective['impressions']) 
pivot_noobjective['CPC'] = (pivot_noobjective['cost_usd'] / pivot_noobjective['clicks']) 
pivot_noobjective['CPS'] = (pivot_noobjective['cost_usd'] / pivot_noobjective['signups']) 
pivot_noobjective['CPP'] = (pivot_noobjective['cost_usd'] / pivot_noobjective['payments']) 
pivot_noobjective['cr2'] = (pivot_noobjective['signups'] / pivot_noobjective['clicks'])
pivot_noobjective['cr3'] = (pivot_noobjective['payments'] / pivot_noobjective['signups'])


,device_type,channel_name,cost_usd,impressions,clicks,signups,payments,label,CPM,CTR,CPC,CPS,CPP,cr2,cr3
0,Android,Facebook News Feed,947.778946,241666,6720,406,311,Facebook News Feed | Android,3.921855,0.027807,0.141039,2.334431,3.047521,0.060417,0.766010
1,Android,Google Search Engine,2068.786097,439756,19249,1457,571,Google Search Engine | Android,4.704395,0.043772,0.107475,1.419894,3.623093,0.075692,0.391901
2,Android,Instagram Stories,500.511884,150865,2929,142,140,Instagram Stories | Android,3.317614,0.019415,0.170881,3.524732,3.575085,0.048481,0.985915
3,Android,MoPub,227.479220,75487,671,0,0,MoPub | Android,3.013489,0.008889,0.339015,inf,inf,0.000000,NaN
4,Android,TikTok,220.984239,97857,855,0,0,TikTok | Android,2.258236,0.008737,0.258461,inf,inf,0.000000,NaN
5,Android,YouTube,548.585343,146032,2754,101,101,YouTube | Android,3.756610,0.018859,0.199196,5.431538,5.431538,0.036674,1.000000
6,Desktop,Facebook News Feed,227.261037,64014,1556,12,12,Facebook News Feed | Desktop,3.550177,0.024307,0.146055,18.938420,18.938420,0.007712,1.000000
7,Desktop,Google Search Engine,540.836578,156920,4950,202,192,Google Search Engine | Desktop,3.446575,0.031545,0.109260,2.677409,2.816857,0.040808,0.950495
8,Desktop,Instagram Stories,122.856453,58483,650,0,0,Instagram Stories | Desktop,2.100721,0.011114,0.189010,inf,inf,0.000000,NaN
9,Desktop,MoPub,61.115309,28881,145,0,0,MoPub | Desktop,2.116108,0.005021,0.421485,inf,inf,0.000000,NaN


In [45]:
color_palette = ['#51a16a', '#057dcd', '#f29f05', '#e3655b', '#9467bd']
fig = px.pie(campaign_summary, 
             names='objective', 
             values='signups', 
             title='Total Impressions by Campaign Type',
             color_discrete_sequence=color_palette,
             labels={'objective': 'Campaign Type', 'impressions': 'Impressions'})

# Customize the layout for better appearance
fig.update_layout(
    title_text='Total Clicks by Campaign Type',
    title_x=0.5,  
    title_font_size=20,
    uniformtext_mode='show'
)
fig.update_traces(
    textinfo='label+percent',  # Show label and percentage
    textfont_size=14,  # Adjust font size for better readability
    textposition='inside',  # Place text inside the slices
    marker=dict(line=dict(color='white', width=1))  # Add a border for better text visibility
)
fig.show()

In [63]:
campaign_types = pivot_table['objective'].unique()
zmin = pivot_table['CPC'].min()
zmax = pivot_table['CPC'].max()


fig = make_subplots(
    rows=4, cols=1,
    subplot_titles= campaign_types,
    shared_yaxes=True,
    shared_xaxes=True,
    vertical_spacing=0.05)

# Add heatmaps for each campaign type
for i, campaign in enumerate(campaign_types):
  
    campaign_data = pivot_table[pivot_table['objective'] == campaign]
    fig.add_trace(
        go.Heatmap(
        z=campaign_data.CPC,
        x=campaign_data.channel_name,
        y=campaign_data.device_type,
        colorscale='YlGnBu',
        zmin=zmin,  
        zmax=zmax,
        colorbar=dict(title='CPC',lenmode='fraction',thickness=15),
        text=campaign_data['CPC'].values,
        texttemplate="%{text:.2f}",  # Use text labels inside cells
        textfont=dict(size=10)),
        row=1+i, col=1)
fig.update_layout(
    title_text='CPC Heatmap by Device Type and Platform for Each Campaign Type',
    title_x=0.5,  # Center the title
    title_font_size=20,
    font=dict(size=12),
    height=900 ,  # Adjust height for better view
    width=1000 # Adjust width to fit all subplots
    )


fig.print_grid()
fig.show()
    

This is the format of your plot grid:
[ (1,1) x,y   ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [33]:
color_palette = ['#44c0ed', '#ffee93', '#8bd0c2', '#d62728', '#9467bd']
fig = px.bar(pivot_table, 
             x='channel_name', 
             y='payments', 
             facet_col='objective', 
             text=pivot_table['payments'].apply(lambda x: f"{x / 1000:.2f}K"),
             color='device_type',
             color_discrete_sequence=color_palette,
             #title='Impressions by Platform and Device Type for Each Campaign Type',
             labels={'impressions': 'Impressions'},
             height=800,
             width=1200)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace('objective=', ''))) 
fig.update_layout(
    title_text='Payments by Channels and Device Types for Each Campaign Type',
    title_x=0.5,  
    title_font_size=20)

fig.update_traces(
    textposition='inside',  
    textfont_size=8  )


fig.show()

In [82]:
pivot_noobjective

,device_type,channel_name,cost_usd,impressions,clicks,signups,payments,label,CPM,CTR,CPC,CPS,CPP,cr2,cr3
0,Android,Facebook News Feed,947.778946,241666,6720,406,311,Facebook News Feed | Android,3.921855,0.027807,0.141039,2.334431,3.047521,0.060417,0.766010
1,Android,Google Search Engine,2068.786097,439756,19249,1457,571,Google Search Engine | Android,4.704395,0.043772,0.107475,1.419894,3.623093,0.075692,0.391901
2,Android,Instagram Stories,500.511884,150865,2929,142,140,Instagram Stories | Android,3.317614,0.019415,0.170881,3.524732,3.575085,0.048481,0.985915
3,Android,MoPub,227.479220,75487,671,0,0,MoPub | Android,3.013489,0.008889,0.339015,inf,inf,0.000000,NaN
4,Android,TikTok,220.984239,97857,855,0,0,TikTok | Android,2.258236,0.008737,0.258461,inf,inf,0.000000,NaN
5,Android,YouTube,548.585343,146032,2754,101,101,YouTube | Android,3.756610,0.018859,0.199196,5.431538,5.431538,0.036674,1.000000
6,Desktop,Facebook News Feed,227.261037,64014,1556,12,12,Facebook News Feed | Desktop,3.550177,0.024307,0.146055,18.938420,18.938420,0.007712,1.000000
7,Desktop,Google Search Engine,540.836578,156920,4950,202,192,Google Search Engine | Desktop,3.446575,0.031545,0.109260,2.677409,2.816857,0.040808,0.950495
8,Desktop,Instagram Stories,122.856453,58483,650,0,0,Instagram Stories | Desktop,2.100721,0.011114,0.189010,inf,inf,0.000000,NaN
9,Desktop,MoPub,61.115309,28881,145,0,0,MoPub | Desktop,2.116108,0.005021,0.421485,inf,inf,0.000000,NaN


In [84]:

df = pivot_noobjective
#[pivot_table['objective'] == 'Branding']
color_palette = ['#44c0ed', '#4c956c','#ffee93', '#ff7700', '#2d2c5e', '#7e5d50']
channel = pivot_noobjective['channel_name'].unique()
color_mapping = {channel: color_palette[i % len(color_palette)] for i, channel in enumerate(channel)}

#df['label'] = df['channel_name'] + ' | ' + df['device_type']

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df['CPC'],
        y=df['cr2'],
        mode='markers+text',
        marker=dict(
            size=10,  
            color=[color_mapping[channel] for channel in df['channel_name']], 
            showscale=True
        ),
        text=df['label'],  
        textposition='top center',
        textfont=dict(size=8)  
    )
)

fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=0.1311,
        x1=0.1311,
        y0=0,  # Start from the minimum y-value
        y1=0.1,  # End at the maximum y-value
        line=dict(
            color="Red",
            width=2,
            dash="dash"
        ),
        name="CPM Reference Line"
    )
)

fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=0,  # Start from the minimum x-value
        x1=0.5,  # End at the maximum x-value
        y0=0.0601,
        y1=0.0601,
        line=dict(
            color="Red",
            width=2,
            dash="dash"
        ),
        name="CTR Reference Line"
    )
)

# Update layout
fig.update_layout(
    title='Scatter Plot of CPC vs. CR2',
    title_x=0.5,
    xaxis_title='CPC',
    yaxis_title='CR2',
    height=800,
    width=1200
)

fig.show()


In [87]:

df = pivot_noobjective

color_palette = ['#44c0ed', '#4c956c','#ffee93', '#ff7700', '#2d2c5e', '#7e5d50']
channel = pivot_noobjective['channel_name'].unique()
color_mapping = {channel: color_palette[i % len(color_palette)] for i, channel in enumerate(channel)}

#df['label'] = df['channel_name'] + ' | ' + df['device_type']

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df['CPM'],
        y=df['CTR'],
        mode='markers+text',
        marker=dict(
            size=10,  
            color=[color_mapping[channel] for channel in df['channel_name']], 
            showscale=True
        ),
        text=df['label'],  
        textposition='top center',
        textfont=dict(size=8)  
    )
)

fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=3.6015,
        x1=3.6015,
        y0=0,  # Start from the minimum y-value
        y1=0.1,  # End at the maximum y-value
        line=dict(
            color="Red",
            width=2,
            dash="dash"
        ),
        name="CPM Reference Line"
    )
)

fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=0,  # Start from the minimum x-value
        x1=5,  # End at the maximum x-value
        y0=0.0275,
        y1=0.0275,
        line=dict(
            color="Red",
            width=2,
            dash="dash"
        ),
        name="CTR Reference Line"
    )
)

# Update layout
fig.update_layout(
    title='Scatter Plot of CPM vs. CTR',
    title_x=0.5,
    xaxis_title='CPM',
    yaxis_title='CTR',
    height=800,
    width=1200
)

fig.show()


In [94]:

df = pivot_noobjective
#[pivot_table['objective'] == 'Branding']
color_palette = ['#44c0ed', '#4c956c','#ffee93', '#ff7700', '#2d2c5e', '#7e5d50']
channel = pivot_noobjective['channel_name'].unique()
color_mapping = {channel: color_palette[i % len(color_palette)] for i, channel in enumerate(channel)}

#df['label'] = df['channel_name'] + ' | ' + df['device_type']

# Create scatter plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df['CPS'],
        y=df['cr3'],
        mode='markers+text',
        marker=dict(
            size=10,  
            color=[color_mapping[channel] for channel in df['channel_name']], 
            showscale=True
        ),
        text=df['label'],  
        textposition='top center',
        textfont=dict(size=6)  
    )
)

fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=2.1802,
        x1=2.1802,
        y0=0,  # Start from the minimum y-value
        y1=1,  # End at the maximum y-value
        line=dict(
            color="Red",
            width=2,
            dash="dash"
        ),
        name="CPM Reference Line"
    )
)

fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=0,  # Start from the minimum x-value
        x1=125,  # End at the maximum x-value
        y0=0.5499,
        y1=0.5499,
        line=dict(
            color="Red",
            width=2,
            dash="dash"
        ),
        name="CTR Reference Line"
    )
)

# Update layout
fig.update_layout(
    title='Scatter Plot of CPS vs. CR3',
    title_x=0.5,
    xaxis_title='CPS',
    yaxis_title='CR3',
    height=800,
    width=1200
)

fig.show()
